In [ ]:
import nltk
nltk.download('wordnet_ic')


# In[2]:


import pandas as pd
import nltk.corpus
from sematch.semantic.similarity import WordNetSimilarity
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import wordnet as wn
import inflect
import re
import json
import random
import pickle
import pandas as pd
import os

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
with open('databricks_dolly_15k.jsonl') as f:
    data = [json.loads(line) for line in f]

In [ ]:
sentences = []

for i in data:
    sentences.append(i['instruction'])
    sentences.append(i['context'])
    sentences.append(i['response'])

In [ ]:
def remove_special_characters(text):
    # Use regular expression to remove special characters except single and double quotations
    text = re.sub(r'[^a-zA-Z0-9\s\']', '', text)  # Allow single and double quotations
    text = text.replace("\n", "")
    return text

sentences = [remove_special_characters(sentence) for sentence in sentences]
sentences = [sentence for sentence in sentences if sentence.strip()]

In [ ]:
def sentence_to_words(sentence):
    return sentence.split()

In [ ]:
# Lowercases all words currently for simplicity.
def watermark_sentence(orig_sentence):
    # Can add
    sentence = [word.lower() for word in orig_sentence.copy()]
    
    # Get the best synonym starting with the word at the end of the sentence.
    result = get_best_synonym(sentence)
    
    if result is None:
        return None
    
    best_synonym, best_synonym_index = result
    
    # Replace the target word with the synonym.
    sentence[best_synonym_index] = best_synonym
    
    return sentence

In [ ]:
def determine_plural_or_singular(word):
    # Initialize the inflect engine
    p = inflect.engine()
    
    # Get the singular form of the word
    singular_form = p.singular_noun(word)
    
    # Compare the singular form with the original word
    if singular_form == False:
        return "Singular"
    else:
        return "Plural" 

In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def is_stopword(word):
    return word.lower() in stop_words

In [ ]:
def get_all_synonyms(word):
    word = word.lower()
    synonyms = []
    scores = []
    for ss in wn.synsets(word):
        synonyms.extend([lemma.lower() for lemma in ss.lemma_names()])
        for sim in ss.similar_tos():
            synonyms_batch = sim.lemma_names()
            synonyms.extend(synonyms_batch)
    synonyms = set(synonyms)
    if word in synonyms:
        synonyms.remove(word)
    synonyms = [synonym.replace('_',' ') for synonym in synonyms]
    
    for s in synonyms:
        if determine_plural_or_singular(s) != determine_plural_or_singular(word):
            synonyms.remove(s)
    return synonyms

In [ ]:
# Gets the index and synonym that is the highest scored synonym.
def get_best_synonym(sentence):
    
    score_list = []
    word_list = []
    
    #print(len(sentence))
    #print(sentence)
    
    for i in range(len(sentence)):
        word = sentence[i]
        if is_stopword(word):
            word_list.append("none")
            score_list.append(0)
        else:
            all_synonyms = get_all_synonyms(word)

            # Ignore current word if there are no synonyms.
            if len(all_synonyms) == 0: 
                # append 0 score.
                score_list.append(0)
                word_list.append("none")
                continue

            wns = WordNetSimilarity()
            similarity_scores = [wns.word_similarity(word, syn) for syn in all_synonyms]

            # Uncomment the following to see process.
            #print(word)
            #print(all_synonyms)
            #print(similarity_scores)

            max_score = max(similarity_scores)
            best_synonym_for_current_idx = all_synonyms[similarity_scores.index(max_score)]
            
            #print(max_score)
            #print(best_synonym_for_current_idx)

            word_list.append(best_synonym_for_current_idx)
            score_list.append(max_score)
        
    #print(word_list)
    #print(score_list)
    #best_score_overall = max(score_list)
    best_word = []
    
    best_score_overall = [index for index, value in enumerate(score_list) if value == 1]
    
    if best_score_overall == 0:
        print("No available synonyms")
        return None
    
    #best_word_idx = score_list.index(best_score_overall)
    for i in best_score_overall:
        best_word.append(word_list[i])
    
    return best_word, best_score_overall


In [ ]:
marked_data_folder = 'c4_llm_multiple_marked_clean/'
unmarked_data_folder = 'c4_llm_multiple_unmarked_clean/'

In [ ]:
start_num_hundreds = 30 #45
num_hundreds = 10 #573
for i in range(start_num_hundreds + 1, num_hundreds + start_num_hundreds + 1):
    start_range = (i - 1) * 100
    end_range = (i) * 100
    
    print(start_range, end_range)
    
    current_sentences = sentences_token[start_range:end_range]
    #print(current_sentences[0])

    watermarked_sents = []
    unmarked_sents = []
    
    for i in range(len(current_sentences)):
        sentence = current_sentences[i].copy()
        
        if len(sentence) < 5:
            continue
        
        result = watermark_sentence(sentence)
        
        if result is not None:
            watermarked_sents.append(result)
            unmarked_sents.append([word.lower() for word in current_sentences[i]])
            
    with open(unmarked_data_folder + 'unmarked' + str(start_range) + 'To' + str(end_range) + '.pkl', 'wb') as file:
        pickle.dump(unmarked_sents, file)
        
    with open(marked_data_folder + 'watermarked' + str(start_range) + 'To' + str(end_range) + '.pkl', 'wb') as file:
        pickle.dump(watermarked_sents, file)
        
    print('Finish')
